In [1]:
import pandas as pd

In [19]:
dataUrls= ["http://schools.nyc.gov/NR/rdonlyres/19E79FA9-ED99-42BA-B89E-56CB931F2465/0/2014_2015_HS_SQR_Results_2016_04_08.xlsx",
          "http://schools.nyc.gov/NR/rdonlyres/32595FE4-15E0-4DFE-A4F4-2D9AD32ED6D1/0/2015_2016_HS_SQR_Results_2017_01_05.xlsx",
          "http://schools.nyc.gov/NR/rdonlyres/CDB6D687-3DAB-4B0D-8DC7-0EF7784E7C3B/0/201617_HS_SQR_Results_2018_01_22.xlsx"]
dataSheets = ["Summary","Student Achievement","Closing the Achievement Gap"]
summaryColsRegex = "(?<!Trust)Percent (?!Positive|of)|(8)|Economic|DBN|Name|Type"

In [21]:
test = pd.read_excel(io=dataUrls[1],sheet_name = dataSheets,header=1)

In [27]:
pd.__version__

'0.19.2'